In [ ]:
import cv2
import matplotlib
import matplotlib.pyplot as plt


def load(filename: str):
    return cv2.imread(f'data/{filename}')


def save(filename: str, image):
    cv2.imwrite(f'results/{filename}.jpg', image)


#https://stackoverflow.com/questions/28816046/
#displaying-different-images-with-actual-size-in-matplotlib-subplot
def show(image):
    dpi = matplotlib.rcParams['figure.dpi']

    if isinstance(image, str):
        image = plt.imread(f'results/{image}.jpg')

    height, width = image.shape[:2]

    # What size does the figure need to be in inches to fit the image?
    figure_size = width / float(dpi), height / float(dpi)

    # Create a figure of the right size with one axes that takes up the full figure
    fig = plt.figure(figsize=figure_size)
    ax = fig.add_axes([0, 0, 1, 1])

    # Display the image.
    ax.imshow(image, cmap='gray')
    plt.show()


In [ ]:
img = load('fata1.jpeg')
# img = img[0:300, 0:450]
save('resized', img)
# plt.imshow(img)
# plt.show()
show(img)

In [ ]:
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()